### Gridsearch CA

In [5]:
!pip install pycsa

  Created wheel for pycsa: filename=pycsa-0.1.3-py3-none-any.whl size=5068 sha256=980e355d52649439ce6c101743e38feb9a7c270b3b65c3afc4b943c1e4b2227a
  Stored in directory: /home/jovyan/.cache/pip/wheels/10/74/f6/ce9cab7f45aacf003d1901916d0d7a6bc0456c092aba414408
Successfully built pycsa


In [1]:
import sys
sys.path.append('./../')
%load_ext autoreload
%autoreload 2

from src.models.CA_utils import diccionario_lugares,get_distance_matrix, get_inits
from pycsa import CoupledAnnealer
import math
import random


 94459- 6 lugares, Nuevo león
 37831- 30 lugares, Jalisco
 14738- 58 lugares, Ciudad de México

In [2]:
import pandas as pd
df = pd.read_csv("./../data/raw/base.csv",encoding='latin-1')
fv = 37831

In [3]:
lugares = diccionario_lugares(df,fv)
distance_matrix = get_distance_matrix(lugares)

init_state = list(lugares.keys())
init_state = random.shuffle(init_state)

def probe(positions, tgen):
    """
    Swap two cities in the route.

    Note that `tgen` (the generation temperature) is ignored here.
    In general, you can use `tgen` to adjust the variance of
    the probing jumps as the algorithm progress.
    """
    a = random.randint(0, len(positions) - 1)
    b = random.randint(0, len(positions) - 1)
    positions[a], positions[b] = positions[b], positions[a]
    return positions


def target(positions):
    """
    Calculates the length of the route.
    """
    try:
        xrange
    except NameError:
        xrange = range
        
    e = 0
    for i in xrange(len(positions)):
        e += distance_matrix[positions[i-1]][positions[i]]
    return e

./../src/models/CA_utils.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['lugar'] = range(1, len(df2) + 1)


In [4]:
n_annealers = 10  # the number of coupled annealers

init_state = list(lugares.keys())
random.shuffle(init_state)


# Initialize the CSA process.
annealer = CoupledAnnealer(
    target, 
    probe, 
    initial_state=[init_state] * n_annealers,
    steps=200,  # You probably want to set this a lot higher, like 10,000
    processes=1,   # Only use more than 1 process if the target function is costly to compute
    n_annealers=n_annealers,
    tacc_initial=1000.0,
    verbose=1)

# Beging the annealing.
annealer.anneal()

Step      5, Energy 137.6234, Elapsed time 0.00 secs
Updated acceptance temp 950.000000
Updated generation temp 0.010000

Step     10, Energy 137.6234, Elapsed time 0.01 secs
Updated acceptance temp 902.500000
Updated generation temp 0.010000

Step     15, Energy 142.2075, Elapsed time 0.01 secs
Updated acceptance temp 857.375000
Updated generation temp 0.010000

Step     20, Energy 139.1746, Elapsed time 0.01 secs
Updated acceptance temp 814.506250
Updated generation temp 0.010000

Step     25, Energy 139.1746, Elapsed time 0.02 secs
Updated acceptance temp 773.780937
Updated generation temp 0.010000

Step     30, Energy 139.1746, Elapsed time 0.02 secs
Updated acceptance temp 735.091891
Updated generation temp 0.009999

Step     35, Energy 139.1746, Elapsed time 0.02 secs
Updated acceptance temp 698.337296
Updated generation temp 0.009999

Step     40, Energy 139.1746, Elapsed time 0.02 secs
Updated acceptance temp 663.420431
Updated generation temp 0.009999

Step     45, Energy 141.

In [9]:
# Get the best result from all `n_annealers`.
energy, state = annealer.get_best()

# Slide the list of cities until NYC is first.
while state[0] != 'base':
    state = state[1:] + state[:1]

print()
print("%i km route:" % energy)
for city in state:
    print("\t", city)


12 km route:
	 base
	 5
	 3
	 6
	 4
	 1
	 2
